In [1]:
import mujoco
import mujoco.viewer
import numpy as np
import time
import scipy.optimize
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import csv

In [ ]:
class MuscleModel:
    def __init__(self, scale: float, wrist_on: bool = False, model_path: str = "model.xml", emg_muscles = []):
        self.scale = scale
        self.wrist_on = wrist_on
        self.model_path = model_path

        if self.wrist_on:
            self.muscle_names = []
            self.joint_names = []  
        else:
            # self.muscle_names = [
            #     "DELT1", "DELT2", "DELT3", "SUPSP", "INFSP", "SUBSC", "TMIN", "TMAJ",
            #     "PECM1", "PECM2", "PECM3", "LAT1", "LAT2", "LAT3", "CORB", "TRIlong",
            #     "TRIlat", "TRImed", "ANC", "SUP", "BIClong", "BICshort", "BRA", "BRD"
            # ]

            # LAT1 not working for some reason

            self.muscle_names = [
                "DELT1", "DELT2", "DELT3", "SUPSP", "INFSP", "SUBSC", "TMIN", "TMAJ",
                "PECM1", "PECM2", "PECM3", "LAT2", "LAT3", "CORB", "TRIlong",
                "TRIlat", "TRImed", "ANC", "SUP", "BIClong", "BICshort", "BRA", "BRD"
            ]

            self.joint_names = [
                # 'acromioclavicular_r1', 'acromioclavicular_r2', 'acromioclavicular_r3', 
                'shoulder_elv',
                'elv_angle',
                'shoulder_rot', 
                'elbow_flexion',
                # 'pro_sup', 
                # 'flexion', 'deviation', 
                # 'shoulder1_r2',
                # 'sternoclavicular_r2', 'sternoclavicular_r3',
                # 'unrothum_r1', 'unrothum_r2', 'unrothum_r3', 'unrotscap_r2', 'unrotscap_r3'
            ]

        self.model = mujoco.MjModel.from_xml_path(self.model_path)
        self.data = mujoco.MjData(self.model)
        # mujoco.mj_forward(self.model, self.data)
        # self._scale_body_mass() 

        self.muscle_ids = [mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, name) for name in self.muscle_names]
        
        self.act = np.zeros_like(self.data.act)
        
        self.act[self.muscle_ids] = 0.1

        self.elv_plane_id = self.model.joint('shoulder_elv').dofadr
        self.elv_angle_id = self.model.joint('elv_angle').dofadr
        self.rot_id = self.model.joint('shoulder_rot').dofadr
        self.elbow_id = self.model.joint('elbow_flexion').dofadr

        self.prev_jac = []

        self.model.opt.timestep = 0.005

    # def _scale_body_mass(self):
    #     try:
    #         body_id = self.model.body(self.target_body)
    #         original_mass = self.model.body_mass[body_id]
    #         self.model.body_mass[body_id] = original_mass * self.scale
    #         print(f"Scaled mass of '{self.target_body}' from {original_mass} to {self.model.body_mass[body_id]}")
    #     except Exception as e:
    #         print(f"Error scaling body mass: {e}")

    def optimize(self, req_torques: list, emg_act = []):
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state0 = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, state0, spec)

        #assuming act contains all the correct activations from previous instant, enforce emg based activations
        for i in range(len(emg_act)):
            self.act[self.emg_muscle_ids[i]] = emg_act[i]
        
        initial_guess = [self.act[i] for i in self.muscle_ids] #set initial guess to activations from previous step

        self.init_state = {}
        self.init_state['qpos'] = self.data.qpos
        self.init_state['qvel'] = self.data.qvel
        self.init_state['act'] = self.act
        
        def cost_fn(activations):
            return np.sum(activations**2)

        def torque_constraint(activations):
            
            mujoco.mj_setState(self.model, self.data, state0, spec)
            # mujoco.mj_step(self.model, self.data)

            # for i in range(len(self.joint_names)):
            #     self.data.qpos[mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_JOINT, self.joint_names[i])] = qpos[i]
            #     self.data.qvel[mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_JOINT, self.joint_names[i])] = qvel[i]
            
            ctrl = self.data.ctrl
            
            for idx, act in zip(self.muscle_ids, activations):
                ctrl[idx] = act

            self.data.ctrl = ctrl
            for _ in range(2):
                self.data.ctrl = ctrl
                mujoco.mj_step(self.model, self.data)

            elv_plane_torq = self.data.qfrc_actuator[self.elv_plane_id] + self.data.qfrc_passive[self.elv_plane_id] + self.data.qfrc_applied[self.elv_plane_id]
            elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id] + self.data.qfrc_passive[self.elv_angle_id] + self.data.qfrc_applied[self.elv_angle_id]
            shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id] + self.data.qfrc_passive[self.rot_id] + self.data.qfrc_applied[self.rot_id]
            elbow_torq = self.data.qfrc_actuator[self.elbow_id] + self.data.qfrc_passive[self.elbow_id] + self.data.qfrc_applied[self.elbow_id]

            torq = [elv_plane_torq[0], elv_angle_torq[0], shoulder_rot_torq[0], elbow_torq[0]]
            # print(activations)
            # print(torq)
            return req_torques - torq

        constraints = {
            'type': 'eq',
            'fun': torque_constraint,
            'tol': 1e-3
        }

        bounds = [(0, 1)] * len(self.muscle_ids)

        start_t = time.time_ns()
        res = scipy.optimize.minimize(
            fun=cost_fn,
            x0=initial_guess,
            method='SLSQP',
            constraints=[constraints],
            bounds=bounds,
            options={'disp': False, 'maxiter': 100}
        )
        end_t = time.time_ns()
        print(f"Optimization took {(end_t - start_t) / 1e6} ms")

        if not res.success:
            print("Optimization failed:", res.message)

        elv_plane_torq = self.data.qfrc_actuator[self.elv_plane_id] + self.data.qfrc_passive[self.elv_plane_id] + self.data.qfrc_applied[self.elv_plane_id]
        elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id] + self.data.qfrc_passive[self.elv_angle_id] + self.data.qfrc_applied[self.elv_angle_id]
        shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id] + self.data.qfrc_passive[self.rot_id] + self.data.qfrc_applied[self.rot_id]
        elbow_torq = self.data.qfrc_actuator[self.elbow_id] + self.data.qfrc_passive[self.elbow_id] + self.data.qfrc_applied[self.elbow_id]

        torq = [elv_plane_torq[0], elv_angle_torq[0], shoulder_rot_torq[0], elbow_torq[0]]

        return res.x, res.fun, res.success, end_t - start_t, torq
    
    def stiffness(self):
        K_muscle = []
        alpha = 23.4
        muscle_forces = []

        for id in self.muscle_ids:
            f = - self.data.actuator_force[id]
            l = self.data.actuator_length[id]
            km = alpha * f/l
            K_muscle.append(km)
            muscle_forces.append(f)
        
        muscle_forces = np.array(muscle_forces)

        jac = self.data.ten_J
        jac_1 = []
        jac_t1 = []
        joint_id_list = [self.elv_plane_id, self.elv_angle_id, self.rot_id, self.elbow_id]
        dJ = []
        
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state1 = np.empty(size, np.float64)
        mujoco.mj_getState(model.model, model.data, state1, spec)

        for i in range(self.model.na):
            if i in self.muscle_ids:
                jac_1.append([])
                for j in range(self.model.nq):
                    if j in joint_id_list:
                        jac_1[-1].append(jac[i, j])

        for id in joint_id_list:
            self.data.qpos[id] += 0.001
        mujoco.mj_forward(self.model, self.data)
        jac = self.data.ten_J

        for i in range(self.model.na):
            if i in self.muscle_ids:
                jac_t1.append([])
                for j in range(self.model.nq):
                    if j in joint_id_list:
                        jac_t1[-1].append(jac[i, j])
        
        jac_1 = np.array(jac_1)
        jac_t1 = np.array(jac_t1)
        
        dJ = jac_t1.T - jac_1.T
        k_diffterm_col = dJ @ muscle_forces
        k_diffterm = [k_diffterm_col for _ in range(len(joint_id_list))]
        k_diffterm = np.array(k_diffterm)
        k_diffterm = k_diffterm.T

        k_joints = jac_1.T @ np.diag(K_muscle) @ jac_1
        k_joints = k_joints + k_diffterm
        return k_joints
    
    def linearize(self):
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        pre_calc_state = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, pre_calc_state, spec)

        n_steps = 3
        act1 = 0.5
        act2 = 0.2

        for m_id in self.muscle_ids:
            self.data.ctrl[m_id] = act1

        for _ in range(n_steps):
            mujoco.mj_step(self.model, self.data)

        a1 = np.array([self.data.act[m_id] for m_id in self.muscle_ids])
        f1 = np.array([self.data.actuator_force[m_id] for m_id in self.muscle_ids])

        ##### TODO: Add condition to check if could not linearize - by checking act1 - a1

        mujoco.mj_setState(self.model, self.data, pre_calc_state, spec)

        for m_id in self.muscle_ids:
            self.data.ctrl[m_id] = act2

        for _ in range(n_steps):
            mujoco.mj_step(self.model, self.data)

        a2 = np.array([self.data.act[m_id] for m_id in self.muscle_ids])
        f2 = np.array([self.data.actuator_force[m_id] for m_id in self.muscle_ids])

        m = (f2 - f1)/(a2 - a1)
        M = np.diag(m)
        c = f1 - M @ a1
        return M,c

    def optimize_fast(self, req_torques):
        
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state0 = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, state0, spec)

        jac = self.data.ten_J
        jac_1 = []
        joint_id_list = [self.elv_plane_id, self.elv_angle_id, self.rot_id, self.elbow_id]

        for i in range(self.model.na):
            if i in self.muscle_ids:
                jac_1.append([])
                for j in range(self.model.nq):
                    if j in joint_id_list:
                        jac_1[-1].append(jac[i, j])       

        jac_1 = np.array(jac_1)

        initial_guess = [self.act[i] for i in self.muscle_ids] #set initial guess to activations from previous step

        M, c = self.linearize() 

        mujoco.mj_setState(self.model, self.data, state0, spec)

        def cost_fn(activations):
            return np.sum(activations**2)
        
        A = jac_1.T @ M
        C = jac_1.T @ c

        def constraint_fun(activations):
            return A @ activations + C - req_torques
        
        def constraint_jac(activations):
            return A
        
        def objective_grad(activations):
            return 2 * activations

        constraints = {
            'type': 'eq',
            'fun': constraint_fun,
            'jac': constraint_jac,
            'tol': 1e-3
        }

        bounds = [(0, 1)] * len(self.muscle_ids)

        start_t = time.time_ns()
        res = scipy.optimize.minimize(
            fun=cost_fn,
            x0=initial_guess,
            method='SLSQP',
            constraints=[constraints],
            bounds=bounds,
            options={'disp': False, 'maxiter': 100},
            jac= objective_grad
        )
        end_t = time.time_ns()
        print(f"Optimization took {(end_t - start_t) / 1e6} ms")

        if not res.success:
            print("Optimization failed:", res.message)

        elv_plane_torq = self.data.qfrc_actuator[self.elv_plane_id] + self.data.qfrc_passive[self.elv_plane_id] + self.data.qfrc_applied[self.elv_plane_id]
        elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id] + self.data.qfrc_passive[self.elv_angle_id] + self.data.qfrc_applied[self.elv_angle_id]
        shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id] + self.data.qfrc_passive[self.rot_id] + self.data.qfrc_applied[self.rot_id]
        elbow_torq = self.data.qfrc_actuator[self.elbow_id] + self.data.qfrc_passive[self.elbow_id] + self.data.qfrc_applied[self.elbow_id]

        torq = [elv_plane_torq[0], elv_angle_torq[0], shoulder_rot_torq[0], elbow_torq[0]]

        return res.x, res.fun, res.success, end_t - start_t, torq

model_path = "myo_sim/arm/myoarm.xml" 
model = MuscleModel(
    scale=1.0,
    wrist_on=False,
    model_path=model_path,
)


In [9]:
spec = mujoco.mjtState.mjSTATE_INTEGRATION
size = mujoco.mj_stateSize(model.model, spec)
state1 = np.empty(size, np.float64)
mujoco.mj_getState(model.model, model.data, state1, spec)


In [31]:
def find_stiffness(q):
    spec = mujoco.mjtState.mjSTATE_INTEGRATION
    size = mujoco.mj_stateSize(model.model, spec) 
    mujoco.mj_setState(model.model, model.data, state1, spec)
    
    # q = [1.57, 0.7, 0.2, 2]
    torque = [-15,-5,-5,2]
    torque = np.array(torque)

    for i in range(len(model.joint_names)):
        model.data.qpos[model.model.joint(model.joint_names[i]).qposadr] = q[i]
        model.data.qvel[model.model.joint(model.joint_names[i]).qposadr] = 0
        model.data.qacc[model.model.joint(model.joint_names[i]).qposadr] = 0

    mujoco.mj_forward(model.model, model.data)

    spec = mujoco.mjtState.mjSTATE_INTEGRATION
    size = mujoco.mj_stateSize(model.model, spec)
    state0 = np.empty(size, np.float64)
    mujoco.mj_getState(model.model, model.data, state0, spec)

    mujoco.mj_inverse(model.model, model.data)
    
    # torque = np.array([model.data.qfrc_inverse[id][0] for id in [model.elv_plane_id, model.elv_angle_id, model.rot_id, model.elbow_id]])
    # torque[1] = -2
    # torque[3] = 2
        
    activations, err, success, t, achieved_torque = model.optimize_fast(req_torques=torque)
    k = model.stiffness()
    print("Stiffness: ", [k[0,0],k[1,1],k[2,2],k[3,3]])
    print("Achieved torque: ", achieved_torque)
    print("Reqd torque = ", torque)

    # k = np.array([[2.22612592e+01 -1.03751631e+01 -4.63603935e+00  7.32344336e-01],[-1.03751631e+01  5.75668051e+01  2.50795380e+00  2.96334339e-01],[-4.63603935e+00  2.50795380e+00  4.28175375e+00  8.00523337e-03],[ 7.32344336e-01  2.96334339e-01  8.00523337e-03  1.00387523e+01]])
    # k = np.array([[22, -10.3, -4.6, 0.73], [-10.3, 57, 2.5, 0.296], [-4.63, 2.5, 4.28, 0.008], [0.73, 0.296, 0.008, 5*10]])
    mujoco.mj_setState(model.model, model.data, state0, spec)

    ctrl = model.data.ctrl
                
    for idx, act in zip(model.muscle_ids, activations):
        ctrl[idx] = act

    model.data.ctrl = ctrl

    body_name = "lunate"  
    body_id = model.model.body(name=body_name).id

    jacp = np.zeros((3, model.model.nv))  
    jacr = np.zeros((3, model.model.nv))  

    mujoco.mj_jacBodyCom(model.model, model.data, jacp, jacr, body_id)

    joint_id_list = [model.elv_plane_id, model.elv_angle_id, model.rot_id, model.elbow_id]

    J = jacp
    # J = np.vstack((jacp, jacr)) 
    J = J[:, joint_id_list]
    J = J[:,:,0]
    
    J_plus = J.T @ np.linalg.inv(J @ J.T)

    K_end = J_plus.T @ k @ J_plus
    K_end = K_end[:3,:3]
    return K_end, k

In [23]:
sh_arr = [0.5, 0.7, 1.0, 1.2, 1.4, 1.6]
el_arr = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2]

q_arr = []
for i in sh_arr:
    for j in el_arr:
        q_arr.append([1.57, i, 0.2, j])

In [37]:
data = {'Km': [], 'Ke': [], 'el_pos': [], 'wrist_pos': [], 'error': []}

with mujoco.viewer.launch_passive(model.model, model.data) as viewer:
    viewer.opt.frame = mujoco.mjtFrame.mjFRAME_WORLD

    # for _ in range(100):
    #     mujoco.mj_step(model.model, model.data)
    for i in range(len(q_arr)):
        # try:
        k, k_m = find_stiffness(q_arr[i])
        # except:
        #     print('error')
        viewer.sync()
        
        body_name = "lunate"  
        body_id = model.model.body(name=body_name).id
        wrist_pos = model.data.xpos[body_id]
        
        elbow_site = "TRI_site_BRA_side"
        site_id = mujoco.mj_name2id(model.model, mujoco.mjtObj.mjOBJ_SITE, elbow_site)
        elbow_pos = model.data.site_xpos[site_id]

        data['Km'].append(k_m)
        data['Ke'].append(k)
        data['el_pos'].append([elbow_pos[0], elbow_pos[1]])
        data['wrist_pos'].append([wrist_pos[0], wrist_pos[1]])
        

Optimization took 56.631769 ms
Optimization failed: Iteration limit reached
Stiffness:  [405.3233904374518, 135.5550885685418, 10.985229021535407, 27.113801012210367]
Achieved torque:  [-2.78636355420537, 13.858426982581918, -10.041947725200458, -10.250493065556746]
Reqd torque =  [-15  -5  -5   2]
Optimization took 60.578204 ms
Optimization failed: Iteration limit reached
Stiffness:  [414.86874856513, 143.94642187874217, 11.038316734284786, 25.673705637111965]
Achieved torque:  [-2.7016141134344798, 12.269876728489393, -11.092492061750987, -10.1089969456053]
Reqd torque =  [-15  -5  -5   2]
Optimization took 63.425733 ms
Optimization failed: Iteration limit reached
Stiffness:  [422.0489494581953, 151.08191061552648, 11.068472052966442, 23.339105703572333]
Achieved torque:  [-2.7096097893337356, 10.624205145299559, -11.862872381536292, -9.567495720524528]
Reqd torque =  [-15  -5  -5   2]
Optimization took 62.371832 ms
Optimization failed: Iteration limit reached
Stiffness:  [426.699053

In [14]:
pio.renderers.default = 'browser'

fig = go.Figure()
res = 30
t = np.linspace(0, 2*np.pi, res)
x = np.cos(t)
y = np.sin(t)       
circle = np.vstack((x,y))

for i in range(len(data['Km'])):
    k_e = data['Ke'][i]
    eigvals, eigvecs = np.linalg.eigh(k_e[:2, :2])
    ellipsoid = eigvecs @ np.diag(eigvals) @ circle
    ellipsoid[0,:] = (ellipsoid[0,:])/5e4 + data['wrist_pos'][i][0]
    ellipsoid[1,:] = (ellipsoid[1,:])/5e4 + data['wrist_pos'][i][1]
    
    fig.add_trace(go.Scatter(x=ellipsoid[0,:], y=ellipsoid[1,:]))
    fig.add_trace(go.Scatter(x = [0, data['el_pos'][i][0]], y = [0, data['el_pos'][i][1]], mode='lines', line=dict(color='rgba(0,255,0,0.3)', width=0.5)))
    fig.add_trace(go.Scatter(x = [data['el_pos'][i][0], data['wrist_pos'][i][0]], y = [data['el_pos'][i][1], data['wrist_pos'][i][1]], mode='lines', line=dict(color='rgba(0,0,255,0.3)', width=0.5)))

fig.update_layout(
    width= 800,
    height = 800,
    title = "Endpoint ellipses",
    xaxis=dict(scaleanchor = 'y', range = [-0.8, 0.2]),
    yaxis=dict(range = [-0.8, 0.2]),
)

fig.show()


Test for muscle activation finding

In [61]:
model.model.opt.timestep = 0.005
print(model.model.opt.timestep)

0.005


In [62]:
# q = [1.59, 0.5, 0.1, 0.8]
q = [0,0,0,0]

for i in range(len(model.joint_names)):
    model.data.qpos[model.model.joint(model.joint_names[i]).qposadr] = q[i]
    model.data.qvel[model.model.joint(model.joint_names[i]).qposadr] = 0
    model.data.qacc[model.model.joint(model.joint_names[i]).qposadr] = 0

In [16]:
spec = mujoco.mjtState.mjSTATE_INTEGRATION
size = mujoco.mj_stateSize(model.model, spec)
state2 = np.empty(size, np.float64)
mujoco.mj_getState(model.model, model.data, state2, spec)

In [19]:
n = 2
lin_arr = []

mujoco.mj_setState(model.model, model.data, state2, spec)

for m_id in model.muscle_ids:
    model.data.ctrl[m_id] = 0.5

jac = model.data.ten_J
jac_1 = []
joint_id_list = [model.elv_plane_id, model.elv_angle_id, model.rot_id, model.elbow_id]

for i in range(model.model.na):
    if i in model.muscle_ids:
        jac_1.append([])
        for j in range(model.model.nq):
            if j in joint_id_list:
                jac_1[-1].append(jac[i, j])

for _ in range(n):
    mujoco.mj_step(model.model, model.data)

a1 = np.array([model.data.act[m_id] for m_id in model.muscle_ids])
f1 = np.array([model.data.actuator_force[m_id] for m_id in model.muscle_ids])

mujoco.mj_setState(model.model, model.data, state2, spec)

for m_id in model.muscle_ids:
    model.data.ctrl[m_id] = 0.2

for _ in range(n):
    mujoco.mj_step(model.model, model.data)

a2 = np.array([model.data.act[m_id] for m_id in model.muscle_ids])
f2 = np.array([model.data.actuator_force[m_id] for m_id in model.muscle_ids])

m = (f2 - f1)/(a2 - a1)
M = np.diag(m)
c = f1 - M @ a1

print(M, c)

[[ -907.24851737     0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.          -594.66198453     0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.             0.          -163.85203257     0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.             0.             0.        

In [65]:
print(lin_arr)

[[-907.2485173669396, -73.7407762451661], [-594.6619845267828, -562.0053699220548], [-163.8520325666314, 0.0], [-447.82828951457293, -1.8641529748254015], [-2197.3151929969486, 2.2737367544323206e-13], [-1111.4970340430634, 1.1368683772161603e-13], [-535.9563539295151, 5.684341886080802e-14], [-222.00285346288518, 2.842170943040401e-14], [-304.95120631860306, -27.98289098204208], [-172.82920997509046, 0.0], [-4.393396670180941, 4.440892098500626e-16], [-530.7826514081474, 0.0], [-59.671606985063136, 7.105427357601002e-15], [-124.34085730959276, 1.4210854715202004e-14], [-664.5161173914688, 5.684341886080802e-14], [-398.6757038819759, 5.684341886080802e-14], [-268.2329661249884, 2.842170943040401e-14], [-5.393449536787951, 4.440892098500626e-16], [-338.1847676667438, -2.463481002972088], [-326.76699890183863, -102.39195080956586], [-279.28800609933774, -0.002830853025216129], [-946.3954594210647, -192.67738687313067], [-120.49292370495743, -142.1169816246389]]
